# Fashion-MNIST


RNN worflow based on the work of [Aymeric Damien](https://github.com/aymericdamien/TensorFlow-Examples/) and [Sungjoon](https://github.com/sjchoi86/tensorflow-101/blob/master/notebooks/rnn_mnist_simple.ipynb)

Good resource: [How to use Dataset in TensorFlow](https://towardsdatascience.com/how-to-use-dataset-in-tensorflow-c758ef9e4428)

## RNN Overview

<img src="http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/RNN-unrolled.png" alt="nn" style="width: 600px;"/>

References:
- [Long Short Term Memory](http://deeplearning.cs.cmu.edu/pdfs/Hochreiter97_lstm.pdf), Sepp Hochreiter & Jurgen Schmidhuber, Neural Computation 9(8): 1735-1780, 1997.

## System Information

In [0]:
!pip install watermark

In [0]:
%load_ext watermark
%watermark -v -m -p tensorflow,numpy -g

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
CPython 3.6.3
IPython 5.5.0

tensorflow 1.6.0
numpy 1.14.2

compiler   : GCC 7.2.0
system     : Linux
release    : 4.4.111+
machine    : x86_64
processor  : x86_64
CPU cores  : 2
interpreter: 64bit
Git hash   :


In [0]:
# From https://stackoverflow.com/questions/48750199/google-colaboratory-misleading-information-about-its-gpu-only-5-ram-available
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize

import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn't guaranteed
gpu = GPUs[0]

def printm():
  process = psutil.Process(os.getpid())
  print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " I Proc size: " + humanize.naturalsize( process.memory_info().rss))
  print('GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB'.format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))

printm()

Gen RAM Free: 9.0 GB  I Proc size: 4.2 GB
GPU RAM Free: 9678MB | Used: 1761MB | Util  15% | Total 11439MB


In [0]:
from pathlib import Path
import random 
from datetime import datetime

import tensorflow as tf
from tensorflow.contrib import rnn
import numpy as np
import pandas as pd

In [0]:
!nvidia-smi

Mon Apr  2 08:12:02 2018       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 384.111                Driver Version: 384.111                   |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    61W / 149W |   1761MiB / 11439MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
!ps -ef|grep python

root        90    80  0 00:11 ?        00:00:14 /usr/bin/python /usr/local/bin/jupyter-notebook -y --no-browser --log-level=DEBUG --debug --NotebookApp.allow_origin="*" --NotebookApp.log_format="%(message)s" --NotebookApp.token= --Session.key="" --Session.keyfile="" --ContentsManager.untitled_directory="Untitled Folder" --ContentsManager.untitled_file="Untitled File" --ContentsManager.untitled_notebook="Untitled Notebook" --NotebookNotary.algorithm="sha1" --KernelManager.autorestart=True --MultiKernelManager.default_kernel_name="python2" --ip="127.0.0.1" --port=9000 --port-retries=0 --notebook-dir="/content" --NotebookNotary.algorithm=sha256 --NotebookNotary.secret_file=/content/datalab/.config/notary_secret --NotebookApp.base_url=/tun/m/gpu-50aa25b0-cff1-4d08-a893-3da31a333462/
root      1134    90 55 04:49 ?        01:53:23 /usr/bin/python3 -m ipykernel_launcher -f /content/.local/share/jupyter/runtime/kernel-51cd6d5d-2d4d-43a6-8cd5-a7e22403b5f3.json
root      1202    90  0 04:50 ?  

## Prepare Fashon-MNIST
[Data Source](https://www.kaggle.com/zalando-research/fashionmnist/data)

In [0]:
from google.colab import auth
auth.authenticate_user()

# https://cloud.google.com/resource-manager/docs/creating-managing-projects
project_id = 'personal-project-196600 '
!gcloud config set project {project_id}

Updated property [core/project].


In [0]:
# Download the file from a given Google Cloud Storage bucket.
!gsutil cp gs://colab-tmp/fashion-mnist_train.csv .
!gsutil cp gs://colab-tmp/fashion-mnist_test.csv .

Copying gs://colab-tmp/fashion-mnist_train.csv...
| [1 files][126.9 MiB/126.9 MiB]                                                
Operation completed over 1 objects/126.9 MiB.                                    
Copying gs://colab-tmp/fashion-mnist_test.csv...
- [1 files][ 21.2 MiB/ 21.2 MiB]                                                
Operation completed over 1 objects/21.2 MiB.                                     


In [0]:
df_train = pd.read_csv("fashion-mnist_train.csv")
df_test = pd.read_csv("fashion-mnist_test.csv")
print(df_train.shape, df_test.shape)

(60000, 785) (10000, 785)


In [0]:
print(df_train.columns)
print(df_test.columns)

Index(['label', 'pixel1', 'pixel2', 'pixel3', 'pixel4', 'pixel5', 'pixel6',
       'pixel7', 'pixel8', 'pixel9',
       ...
       'pixel775', 'pixel776', 'pixel777', 'pixel778', 'pixel779', 'pixel780',
       'pixel781', 'pixel782', 'pixel783', 'pixel784'],
      dtype='object', length=785)
Index(['label', 'pixel1', 'pixel2', 'pixel3', 'pixel4', 'pixel5', 'pixel6',
       'pixel7', 'pixel8', 'pixel9',
       ...
       'pixel775', 'pixel776', 'pixel777', 'pixel778', 'pixel779', 'pixel780',
       'pixel781', 'pixel782', 'pixel783', 'pixel784'],
      dtype='object', length=785)


In [0]:
# Train/Validation Split
idx = np.arange(60000)
np.random.shuffle(idx)
print(idx[:5])
df_val = df_train.iloc[idx[:10000]]
df_train = df_train.iloc[idx[10000:]]
print(df_val.shape, df_train.shape)

[31325 57677 37900 23176 38365]
(10000, 785) (50000, 785)


## Build the Model

In [0]:
# Training Parameters
learning_rate = 0.002
training_steps = 10000
batch_size = 32
display_step = 500
print("Total batches per epoch:", np.ceil(df_train.shape[0] / batch_size))

# Network Parameters
num_input = 1 # MNIST data input (img shape: 28*28)
timesteps = 28 * 28 # timesteps
num_hidden = 128 # hidden layer num of features
num_classes = 10 # MNIST total classes (0-9 digits)

Total batches per epoch: 1563.0


In [0]:
def RNN(x):
    # Define a lstm cell with tensorflow
    gru = tf.contrib.cudnn_rnn.CudnnGRU(
            1, num_hidden,
            # kernel_initializer=tf.random_normal_initializer(0, 1))
            kernel_initializer=tf.orthogonal_initializer())
            # kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=True))
            # kernel_initializer=tf.truncated_normal_initializer())

    # Get lstm cell output
    # outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)
    outputs, _ = gru(tf.transpose(x, (1, 0, 2)))
    # return tf.matmul(outputs[-1, :, :], weights['out']) + biases['out']
    output_layer = tf.layers.Dense(
        num_classes, activation=None, 
        # kernel_initializer=tf.random_normal_initializer(0, 1),
        kernel_initializer=tf.orthogonal_initializer(),
        # kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=True),
        # kernel_initializer=tf.glorot_uniform_initializer(),
        trainable=True
    )
    # Linear activation, using rnn inner loop last output
    return output_layer(tf.layers.batch_normalization(outputs[-1, :, :]))
    # return output_layer(outputs[-1, :, :])

In [0]:
def process_batch(batch_x, batch_y):
    return tf.expand_dims(batch_x, -1), tf.one_hot(batch_y, num_classes)

In [0]:
tf.reset_default_graph()
graph = tf.Graph()
with graph.as_default():
    tf.set_random_seed(10)
    
    with tf.variable_scope("datasets"):
        training_batch_size = tf.placeholder(tf.int64) # tf.constant(32, dtype="int64")
        inference_batch_size = tf.placeholder(tf.int64) # tf.constant(500, dtype="int64")

        fminst_ds_train = tf.data.Dataset.from_tensor_slices(
            (df_train.iloc[:, 1:].astype("float32") / 255, df_train.iloc[:, 0].astype("int32"))
        ).shuffle(50000, reshuffle_each_iteration=True).repeat().batch(training_batch_size).map(process_batch)
        fminst_ds_val = tf.data.Dataset.from_tensor_slices(
            (df_val.iloc[:, 1:].astype("float32") / 255, df_val.iloc[:, 0].astype("int32"))
        ).repeat().batch(inference_batch_size).map(process_batch)
        fminst_ds_test = tf.data.Dataset.from_tensor_slices((
            df_test.iloc[:, 1:].astype("float32") / 255, df_test.iloc[:, 0].astype("int32"))
        ).repeat().batch(inference_batch_size).map(process_batch)

        handle = tf.placeholder(tf.string, shape=[])
        iterator = tf.data.Iterator.from_string_handle(
            handle, fminst_ds_train.output_types, fminst_ds_train.output_shapes)

        train_iterator = fminst_ds_train.make_initializable_iterator()
        val_iterator = fminst_ds_val.make_initializable_iterator()
        test_iterator = fminst_ds_test.make_initializable_iterator() 
    
    X_0, Y = iterator.get_next()
    X = tf.reshape(X_0, (-1, timesteps, num_input))

    # Define weights
    logits = RNN(X)
    prediction = tf.nn.softmax(logits)
   
    with tf.name_scope("loss"):
        # Define loss and optimizer
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
            logits=logits, labels=Y))
        ema_loss = tf.get_variable("ema_loss", shape=[], dtype=tf.float32, trainable=False, initializer=tf.constant_initializer(2.5))
        ema_update = ema_loss.assign(ema_loss * 0.99 + loss * 0.01)
    tf.summary.scalar('Loss', ema_loss)
    
    with tf.variable_scope("optimizer"):
        # optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate)
        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=0.1)
        gvs = optimizer.compute_gradients(loss)
        capped_gvs = [(tf.clip_by_norm(grad, 1.), var) for grad, var in gvs]
        with tf.control_dependencies([ema_update]):
            train_op = optimizer.apply_gradients(capped_gvs)    
        # train_op = optimizer.minimize(loss_op)

    with tf.variable_scope("inference"):
        # Evaluate model (with test logits, for dropout to be disabled)
        correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

    with tf.variable_scope('summarize_gradients'):
        for grad, var in gvs:
            norm = tf.norm(tf.clip_by_norm(grad, 10.), ord=2)
            tf.summary.histogram(var.name.replace(":", "_") + '/gradient_l2', 
                                 tf.where(tf.is_nan(norm), tf.zeros_like(norm), norm))
        for grad, var in capped_gvs:
            norm = tf.norm(grad, ord=2)
            tf.summary.histogram(var.name.replace(":", "_") + '/gradient_clipped_l2', 
                                 tf.where(tf.is_nan(norm), tf.zeros_like(norm), norm)) 
            
    # Initialize the variables (i.e. assign their default value)
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()
    merged_summary_op = tf.summary.merge_all()
    # print([x.get_shape() for x in tf.global_variables()])
    # print([x.get_shape() for x in tf.trainable_variables()])
    # print("All parameters:", np.sum([np.product([xi.value for xi in x.get_shape()]) for x in tf.global_variables()]))
    # print("Trainable parameters:", np.sum([np.product([xi.value for xi in x.get_shape()]) for x in tf.trainable_variables()]))

### Training and Evaluating

In [0]:
# Start training
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
best_val_acc = 0.8

train_writer = tf.summary.FileWriter("logs/fminst_gru/%s/train" % datetime.now().strftime("%Y%m%d_%H%M"), graph)
val_writer = tf.summary.FileWriter("logs/fminst_gru/%s/val" % datetime.now().strftime("%Y%m%d_%H%M"))

with tf.Session(graph=graph, config=config) as sess:
    sess.run(init) 
    sess.run([train_iterator.initializer, val_iterator.initializer, test_iterator.initializer],
             feed_dict={training_batch_size: batch_size, inference_batch_size: 500})
    train_handle, val_handle, test_handle = sess.run(
        [train_iterator.string_handle(), val_iterator.string_handle(), test_iterator.string_handle()])
    # Run the initializer
    for step in range(1, training_steps+1):
        sess.run([train_op], feed_dict={handle: train_handle})
        
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss_local, acc, summary = sess.run([loss, accuracy, merged_summary_op], feed_dict={handle: train_handle})
            train_writer.add_summary(summary, global_step=step)
            train_writer.flush()            
            val_acc, val_loss = [], []
            for _ in range(20):
                tmp = sess.run([accuracy, loss], feed_dict={handle: val_handle})
                val_acc.append(tmp[0])
                val_loss.append(tmp[1])
            summary = tf.Summary()
            val_loss = np.mean(val_loss)            
            val_acc = np.mean(val_acc)            
            summary.value.add(tag='Loss', simple_value=val_loss)
            val_writer.add_summary(summary, global_step=step)
            val_writer.flush() 
            print("Step " + str(step) + ", Train Loss= " + \
                  "{:.4f}".format(loss_local) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc) + ", Val Accuracy= " + \
                  "{:.3f}".format(val_acc))
            if val_acc > best_val_acc:
                best_val_acc = val_acc
                save_path = saver.save(sess, "/tmp/model.ckpt")
                print("Model saved in path: %s" % save_path)  
    test_acc = []
    for _ in range(20):
        test_acc.append(sess.run(accuracy, feed_dict={handle: test_handle}))
    test_acc = np.mean(test_acc)
    print("Test Accuracy= {:.3f}".format(test_acc))                
    print("Optimization Finished!")
    
train_writer.close()
val_writer.close()

Step 1, Train Loss= 2.3047, Training Accuracy= 0.125, Val Accuracy= 0.118
Step 500, Train Loss= 1.3528, Training Accuracy= 0.469, Val Accuracy= 0.469
Step 1000, Train Loss= 1.1724, Training Accuracy= 0.594, Val Accuracy= 0.497
Step 1500, Train Loss= 0.6783, Training Accuracy= 0.781, Val Accuracy= 0.720
Step 2000, Train Loss= 0.2735, Training Accuracy= 0.969, Val Accuracy= 0.778
Step 2500, Train Loss= 0.3495, Training Accuracy= 0.875, Val Accuracy= 0.798


In [0]:
with tf.Session(graph=graph, config=config) as sess:
    saver.restore(sess, '/tmp/model.ckpt')
    sess.run(test_iterator.initializer,
             feed_dict={inference_batch_size: 500})
    test_handle = sess.run(test_iterator.string_handle())    
    test_acc = []
    for _ in range(20):
        test_acc.append(sess.run(accuracy, feed_dict={handle: test_handle}))
    test_acc = np.mean(test_acc)
    print("Test Accuracy= {:.3f}".format(test_acc))

INFO:tensorflow:Restoring parameters from /tmp/model.ckpt
Test Accuracy= 0.887


## Permute

In [0]:
# Training Parameters
training_steps = 15000

In [0]:
tf.reset_default_graph()
graph = tf.Graph()
with graph.as_default():
    tf.set_random_seed(10)
    
    with tf.variable_scope("datasets"):
        training_batch_size = tf.placeholder(tf.int64) # tf.constant(32, dtype="int64")
        inference_batch_size = tf.placeholder(tf.int64) # tf.constant(500, dtype="int64")

        fminst_ds_train = tf.data.Dataset.from_tensor_slices(
            (df_train.iloc[:, 1:].astype("float32") / 255, df_train.iloc[:, 0].astype("int32"))
        ).shuffle(50000, reshuffle_each_iteration=True).repeat().batch(training_batch_size).map(process_batch)
        fminst_ds_val = tf.data.Dataset.from_tensor_slices(
            (df_val.iloc[:, 1:].astype("float32") / 255, df_val.iloc[:, 0].astype("int32"))
        ).repeat().batch(inference_batch_size).map(process_batch)
        fminst_ds_test = tf.data.Dataset.from_tensor_slices((
            df_test.iloc[:, 1:].astype("float32") / 255, df_test.iloc[:, 0].astype("int32"))
        ).repeat().batch(inference_batch_size).map(process_batch)

        handle = tf.placeholder(tf.string, shape=[])
        iterator = tf.data.Iterator.from_string_handle(
            handle, fminst_ds_train.output_types, fminst_ds_train.output_shapes)

        train_iterator = fminst_ds_train.make_initializable_iterator()
        val_iterator = fminst_ds_val.make_initializable_iterator()
        test_iterator = fminst_ds_test.make_initializable_iterator() 
    
    X_0, Y = iterator.get_next()
    
    # Permute the time step
    np.random.seed(100)
    permute = np.random.permutation(784)
    X = tf.gather(
        tf.reshape(X_0, (-1, timesteps, num_input)), 
        permute, axis=1)
    
    # Define weights
    logits = RNN(X)
    prediction = tf.nn.softmax(logits)
   
    with tf.name_scope("loss"):
        # Define loss and optimizer
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
            logits=logits, labels=Y))
        ema_loss = tf.get_variable("ema_loss", shape=[], dtype=tf.float32, trainable=False, initializer=tf.constant_initializer(2.5))
        ema_update = ema_loss.assign(ema_loss * 0.99 + loss * 0.01)
    tf.summary.scalar('Loss', ema_loss)
    
    with tf.variable_scope("optimizer"):
        # optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate)
        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=0.1)
        gvs = optimizer.compute_gradients(loss)
        capped_gvs = [(tf.clip_by_norm(grad, 1.), var) for grad, var in gvs]
        with tf.control_dependencies([ema_update]):
            train_op = optimizer.apply_gradients(capped_gvs)    
        # train_op = optimizer.minimize(loss_op)

    with tf.variable_scope("inference"):
        # Evaluate model (with test logits, for dropout to be disabled)
        correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

    with tf.variable_scope('summarize_gradients'):
        for grad, var in gvs:
            norm = tf.norm(tf.clip_by_norm(grad, 10.), ord=2)
            tf.summary.histogram(var.name.replace(":", "_") + '/gradient_l2', 
                                 tf.where(tf.is_nan(norm), tf.zeros_like(norm), norm))
        for grad, var in capped_gvs:
            norm = tf.norm(grad, ord=2)
            tf.summary.histogram(var.name.replace(":", "_") + '/gradient_clipped_l2', 
                                 tf.where(tf.is_nan(norm), tf.zeros_like(norm), norm)) 
            
    # Initialize the variables (i.e. assign their default value)
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()
    merged_summary_op = tf.summary.merge_all()
    # print([x.get_shape() for x in tf.global_variables()])
    # print([x.get_shape() for x in tf.trainable_variables()])
    # print("All parameters:", np.sum([np.product([xi.value for xi in x.get_shape()]) for x in tf.global_variables()]))
    # print("Trainable parameters:", np.sum([np.product([xi.value for xi in x.get_shape()]) for x in tf.trainable_variables()]))

In [0]:
# Start training
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
best_val_acc = 0.8

train_writer = tf.summary.FileWriter("logs/fminst_gru/%s/train" % datetime.now().strftime("%Y%m%d_%H%M"), graph)
val_writer = tf.summary.FileWriter("logs/fminst_gru/%s/val" % datetime.now().strftime("%Y%m%d_%H%M"))

with tf.Session(graph=graph, config=config) as sess:
    sess.run(init) 
    sess.run([train_iterator.initializer, val_iterator.initializer, test_iterator.initializer],
             feed_dict={training_batch_size: batch_size, inference_batch_size: 500})
    train_handle, val_handle, test_handle = sess.run(
        [train_iterator.string_handle(), val_iterator.string_handle(), test_iterator.string_handle()])
    # Run the initializer
    for step in range(1, training_steps+1):
        sess.run([train_op], feed_dict={handle: train_handle})
        
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss_local, acc, summary = sess.run([loss, accuracy, merged_summary_op], feed_dict={handle: train_handle})
            train_writer.add_summary(summary, global_step=step)
            train_writer.flush()            
            val_acc, val_loss = [], []
            for _ in range(20):
                tmp = sess.run([accuracy, loss], feed_dict={handle: val_handle})
                val_acc.append(tmp[0])
                val_loss.append(tmp[1])
            summary = tf.Summary()
            val_loss = np.mean(val_loss)            
            val_acc = np.mean(val_acc)            
            summary.value.add(tag='Loss', simple_value=val_loss)
            val_writer.add_summary(summary, global_step=step)
            val_writer.flush() 
            print("Step " + str(step) + ", Train Loss= " + \
                  "{:.4f}".format(loss_local) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc) + ", Val Accuracy= " + \
                  "{:.3f}".format(val_acc))
            if val_acc > best_val_acc:
                best_val_acc = val_acc
                save_path = saver.save(sess, "/tmp/model.ckpt")
                print("Model saved in path: %s" % save_path)  
    test_acc = []
    for _ in range(20):
        test_acc.append(sess.run(accuracy, feed_dict={handle: test_handle}))
    test_acc = np.mean(test_acc)
    print("Test Accuracy= {:.3f}".format(test_acc))                
    print("Optimization Finished!")
    
train_writer.close()
val_writer.close()

Step 1, Train Loss= 2.3060, Training Accuracy= 0.031, Val Accuracy= 0.088
Step 500, Train Loss= 1.2360, Training Accuracy= 0.594, Val Accuracy= 0.592
Step 1000, Train Loss= 1.0488, Training Accuracy= 0.688, Val Accuracy= 0.669
Step 1500, Train Loss= 0.6931, Training Accuracy= 0.750, Val Accuracy= 0.717
Step 2000, Train Loss= 0.5201, Training Accuracy= 0.844, Val Accuracy= 0.755
Step 2500, Train Loss= 0.4615, Training Accuracy= 0.906, Val Accuracy= 0.768
Step 3000, Train Loss= 0.4192, Training Accuracy= 0.844, Val Accuracy= 0.772
Step 3500, Train Loss= 0.7379, Training Accuracy= 0.750, Val Accuracy= 0.781
Step 4000, Train Loss= 0.3400, Training Accuracy= 0.875, Val Accuracy= 0.789
Step 4500, Train Loss= 0.3499, Training Accuracy= 0.906, Val Accuracy= 0.800
Model saved in path: /tmp/model.ckpt
Step 5000, Train Loss= 0.6448, Training Accuracy= 0.781, Val Accuracy= 0.800
Model saved in path: /tmp/model.ckpt
Step 5500, Train Loss= 0.5430, Training Accuracy= 0.781, Val Accuracy= 0.797
Step 6

In [0]:
with tf.Session(graph=graph, config=config) as sess:
    saver.restore(sess, '/tmp/model.ckpt')
    sess.run(test_iterator.initializer,
             feed_dict={inference_batch_size: 500})
    test_handle = sess.run(test_iterator.string_handle())    
    test_acc = []
    for _ in range(20):
        test_acc.append(sess.run(accuracy, feed_dict={handle: test_handle}))
    test_acc = np.mean(test_acc)
    print("Test Accuracy= {:.3f}".format(test_acc))

INFO:tensorflow:Restoring parameters from /tmp/model.ckpt
Test Accuracy= 0.850
